In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import argparse

In [ ]:
# Universe size - 100 x 100
# no of universe iterations - 50
# interval - 300 ms
# seed - seed for life
# seed positon - x , y

In [ ]:
infinite_seed = [
    [1, 1, 1, 0, 1],
    [1, 0, 0, 0, 0],
    [0, 0, 0, 1, 1],
    [0, 1, 1, 0, 1],
    [1, 0, 1, 0, 1],
]

In [ ]:
def life_cycle(universe_size=(100,100), seed=infinite_seed, seed_pos=(40, 40), n_generations=10, interval=300):
    # making the visual universe
    universe = np.zeros(universe_size)
    x_start, y_start = seed_pos[0], seed_pos[1]
    seed_array = np.array(seed)
    x_end, y_end = x_start + seed_array.shape[0], y_start + seed_array.shape[1]
    # putting the life seed in the universe
    universe[x_start:x_end, y_start:y_end] = seed_array
    
    # Life loop
    fig = plt.figure(dpi=200)
    plt.axis("off")
    ims = []
    for i in range(n_generations):
        ims.append((plt.imshow(universe),))
        universe = generation(universe)

def generation(universe):
    nw_univ = np.copy(universe)
    # apply the survival function to every cell in the universe
    for i in range(universe.shape[0]):
        for j in range(universe.shape[1]):
            nw_univ[i, j] = survival(i, j, universe)
    return nw_univ

# The GOL's rule of life
def survival(x, y, universe):
    num_neighbours = np.sum(universe[x - 1: x + 2, y - 1 : y + 2]) - universe[x, y]
    # All live cells with no 2 or 3 neighbours die in the next gen
    if universe[x, y] and not 2 <= num_neighbours <= 3:
        return 0
    # Dead cell with 3 neighbours comes to life
    elif num_neighbours == 3:
        return 1
    # Living cell with 2 or 3 live neighbours survives
    return universe[x, y]